In [1]:
import pandas as pd
from konlpy.tag import Kkma
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from collections import Counter
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import transformers
from konlpy.tag import Mecab
from collections import Counter

In [2]:
# matplotlib 한글 패치
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
    rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin': # Mac
    rc('font', family='AppleGothic')
else: #linux
    rc('font', family='NanumGothic')

In [3]:
# .. 상위 폴더
train_pd = pd.read_csv('../../../../data/train.csv')
train_pd

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0
...,...,...,...,...,...,...
9319,boostcamp-sts-v1-train-9319,petition-sampled,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요,0.2,0.0
9320,boostcamp-sts-v1-train-9320,petition-sampled,여성가족부의 폐지를 원합니드,여성가족부 폐지를 청원 합니다.,4.2,1.0
9321,boostcamp-sts-v1-train-9321,petition-sampled,국회의원들 월급좀 줄여주세요,공무원 봉급좀 줄이지좀 마세요,0.6,0.0
9322,boostcamp-sts-v1-train-9322,slack-sampled,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!!,오늘 못먹은 밥은 꼭 담에 먹기로 하고요!!,3.2,1.0


In [4]:
# apply 함수로 일괄 적용

# sentence 합치기
train_pd['sentence_a'] = train_pd['sentence_1'] + ' ' + train_pd['sentence_2']

# 공백 기준으로 토큰 나누기
train_pd['tokens_a']=train_pd['sentence_a'].apply(lambda x: x.split())
train_pd['tokens_1']=train_pd['sentence_1'].apply(lambda x: x.split())
train_pd['tokens_2']=train_pd['sentence_2'].apply(lambda x: x.split())

# 문장 길이, 토큰 개수 칼럼 만들기
train_pd['sen_len_a']=train_pd['sentence_a'].apply(lambda x: len(x))
train_pd['sen_len_1']=train_pd['sentence_1'].apply(lambda x: len(x))
train_pd['sen_len_2']=train_pd['sentence_2'].apply(lambda x: len(x))

train_pd['tok_count_a']=train_pd['tokens_a'].apply(lambda x: len(x))
train_pd['tok_count_1']=train_pd['tokens_1'].apply(lambda x: len(x))
train_pd['tok_count_2']=train_pd['tokens_2'].apply(lambda x: len(x))

# label 단순화, 소숫점 버림
train_pd['simple_label'] = train_pd['label'].apply(lambda x: int(x))

train_pd.head()

,id,source,sentence_1,sentence_2,label,binary-label,sentence_a,tokens_a,tokens_1,tokens_2,sen_len_a,sen_len_1,sen_len_2,tok_count_a,tok_count_1,tok_count_2,simple_label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0,"스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~ 반전도 있고,...","[스릴도있고, 반전도, 있고, 여느, 한국영화, 쓰레기들하고는, 차원이, 다르네요~...","[스릴도있고, 반전도, 있고, 여느, 한국영화, 쓰레기들하고는, 차원이, 다르네요~]","[반전도, 있고,사랑도, 있고재미도있네요.]",59,38,20,11,8,3,2
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0,"앗 제가 접근권한이 없다고 뜹니다;; 오, 액세스 권한이 없다고 합니다.","[앗, 제가, 접근권한이, 없다고, 뜹니다;;, 오,, 액세스, 권한이, 없다고, ...","[앗, 제가, 접근권한이, 없다고, 뜹니다;;]","[오,, 액세스, 권한이, 없다고, 합니다.]",40,20,19,10,5,5,4
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0,주택청약조건 변경해주세요. 주택청약 무주택기준 변경해주세요.,"[주택청약조건, 변경해주세요., 주택청약, 무주택기준, 변경해주세요.]","[주택청약조건, 변경해주세요.]","[주택청약, 무주택기준, 변경해주세요.]",33,14,18,5,2,3,2
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0,입사후 처음 대면으로 만나 반가웠습니다. 화상으로만 보다가 리얼로 만나니 정말 반가...,"[입사후, 처음, 대면으로, 만나, 반가웠습니다., 화상으로만, 보다가, 리얼로, ...","[입사후, 처음, 대면으로, 만나, 반가웠습니다.]","[화상으로만, 보다가, 리얼로, 만나니, 정말, 반가웠습니다.]",51,22,28,11,5,6,3
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0,뿌듯뿌듯 하네요!! 꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,"[뿌듯뿌듯, 하네요!!, 꼬옥, 실제로, 한번, 뵈어요, 뿌뿌뿌~!~!]","[뿌듯뿌듯, 하네요!!]","[꼬옥, 실제로, 한번, 뵈어요, 뿌뿌뿌~!~!]",32,10,21,7,2,5,0


In [5]:
tokenizer = transformers.AutoTokenizer.from_pretrained('klue/roberta-small')
df = train_pd.drop(columns=['id'])
data = []
for idx, item in tqdm(df.iterrows(), desc='tokenizing', total=len(df)):
# 두 입력 문장을 [SEP] 토큰으로 이어붙여서 전처리합니다.
    text = '[SEP]'.join([item[text_column] for text_column in ['sentence_1', 'sentence_2']])
    outputs = tokenizer(text, add_special_tokens=True, padding='do_not_pad', truncation=True)
    data.append(outputs['input_ids'])

tokenizing:   0%|          | 0/9324 [00:00<?, ?it/s]

In [6]:
' '.join(tokenizer.batch_decode(data[1]))

'[CLS] 앗 제 ##가 접근 ##권 ##한 ##이 없 ##다고 [UNK] ; ; [SEP] 오 , 액세 ##스 권한 ##이 없 ##다고 합니다 . [SEP]'

In [7]:
data[1]

[0,
 1385,
 1545,
 2116,
 4872,
 2207,
 2470,
 2052,
 1415,
 4683,
 3,
 31,
 31,
 2,
 1443,
 16,
 12700,
 2255,
 5851,
 2052,
 1415,
 4683,
 3803,
 18,
 2]

In [8]:
samp_pd = train_pd[['sentence_1','sentence_2','sentence_a','label']]
samp_pd

,sentence_1,sentence_2,sentence_a,label
0,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.","스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~ 반전도 있고,...",2.2
1,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.","앗 제가 접근권한이 없다고 뜹니다;; 오, 액세스 권한이 없다고 합니다.",4.2
2,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,주택청약조건 변경해주세요. 주택청약 무주택기준 변경해주세요.,2.4
3,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,입사후 처음 대면으로 만나 반가웠습니다. 화상으로만 보다가 리얼로 만나니 정말 반가...,3.0
4,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,뿌듯뿌듯 하네요!! 꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0
...,...,...,...,...
9319,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다 본인이 납부한 국...,0.2
9320,여성가족부의 폐지를 원합니드,여성가족부 폐지를 청원 합니다.,여성가족부의 폐지를 원합니드 여성가족부 폐지를 청원 합니다.,4.2
9321,국회의원들 월급좀 줄여주세요,공무원 봉급좀 줄이지좀 마세요,국회의원들 월급좀 줄여주세요 공무원 봉급좀 줄이지좀 마세요,0.6
9322,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!!,오늘 못먹은 밥은 꼭 담에 먹기로 하고요!!,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!! 오늘 못먹은 밥은 꼭 담에 먹...,3.2


In [9]:
kkma = Kkma()
' '.join(kkma.morphs(samp_pd['sentence_1'][0]))

'스릴 도 있 고 반전 도 있 고 여느 한국 영화 쓰레기 들 하 고는 차원 이 다르 네요 ~'

In [10]:
kkma = Kkma()
from numpy import NaN
dl1 = []
dl2 = []
for i in range(len(samp_pd)):
    try:
        dl1.append(' '.join(kkma.morphs(samp_pd['sentence_1'][i])))
    except:
        dl1.append(samp_pd['sentence_1'][i])
    try:
        dl2.append(' '.join(kkma.morphs(samp_pd['sentence_2'][i])))
    except:
        dl2.append(samp_pd['sentence_2'][i])
samp_pd['pos_sen_1'] = dl1
samp_pd['pos_sen_2'] = dl2

c:\Users\kzvxy\.conda\envs\test\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\kzvxy\.conda\envs\test\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [11]:
samp_pd

,sentence_1,sentence_2,sentence_a,label,pos_sen_1,pos_sen_2
0,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.","스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~ 반전도 있고,...",2.2,스릴 도 있 고 반전 도 있 고 여느 한국 영화 쓰레기 들 하 고는 차원 이 다르 ...,"반전 도 있 고 , 사랑 도 있 고 재미 도 있 네요 ."
1,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.","앗 제가 접근권한이 없다고 뜹니다;; 오, 액세스 권한이 없다고 합니다.",4.2,앗 저 가 접근 권한 이 없다 고 뜨 ㅂ니다 ;;,"오 , 액세스 권한 이 없다 고 하 ㅂ니다 ."
2,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,주택청약조건 변경해주세요. 주택청약 무주택기준 변경해주세요.,2.4,주택 청약 조건 변경 하 어 주 세요 .,주택 청약 무 주택 기준 변경 하 어 주 세요 .
3,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,입사후 처음 대면으로 만나 반가웠습니다. 화상으로만 보다가 리얼로 만나니 정말 반가...,3.0,입사 후 처음 대면 으로 만 나 반갑 었 습니다 .,화상 으로 만 보 다가 리얼 로 만나 니 정말 반갑 었 습니다 .
4,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,뿌듯뿌듯 하네요!! 꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,뿌듯 뿌듯 하 네요 !!,꼬옥 실제로 한번 뵈 어요 뿌뿌뿌 ~ ! ~ !
...,...,...,...,...,...,...
9319,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다 본인이 납부한 국...,0.2,교원 능력 개발 평가 에서 교원 이 보호 받 을 수 있 는 장치 를 마련 하 어야 ...,본인 이 납부 하 ㄴ 국민 연금 금액 을 기준 으로 대출 을 받 을 수 있 는 제도...
9320,여성가족부의 폐지를 원합니드,여성가족부 폐지를 청원 합니다.,여성가족부의 폐지를 원합니드 여성가족부 폐지를 청원 합니다.,4.2,여성 가족 부의 폐지 를 원 합 니 드,여성 가족 부 폐지 를 청원 하 ㅂ니다 .
9321,국회의원들 월급좀 줄여주세요,공무원 봉급좀 줄이지좀 마세요,국회의원들 월급좀 줄여주세요 공무원 봉급좀 줄이지좀 마세요,0.6,국회의원 들 월급 좀 줄이 어 주 세요,공무원 봉급 좀 줄이 지 좀 마 세요
9322,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!!,오늘 못먹은 밥은 꼭 담에 먹기로 하고요!!,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!! 오늘 못먹은 밥은 꼭 담에 먹...,3.2,오늘 못하 ㄴ 점심 은 다음 에 다시 츄 라이 하 기로 해 요 !!,오늘 못 먹 은 밥 은 꼭 담 에 먹 기로 하 고요 !!


In [12]:
df = samp_pd
data2 = []
for idx, item in tqdm(df.iterrows(), desc='tokenizing', total=len(df)):
# 두 입력 문장을 [SEP] 토큰으로 이어붙여서 전처리합니다.
    text = '[SEP]'.join([item[text_column] for text_column in ['pos_sen_1', 'pos_sen_2']])
    outputs = tokenizer(text, add_special_tokens=True, padding='do_not_pad', truncation=True)
    data2.append(outputs['input_ids'])

tokenizing:   0%|          | 0/9324 [00:00<?, ?it/s]

In [13]:
' '.join(tokenizer.batch_decode(data2[1][:150]))

'[CLS] 앗 저 가 접근 권한 이 없 ##다 고 뜨 ㅂ ##니다 ; ; [SEP] 오 , 액세 ##스 권한 이 없 ##다 고 하 ㅂ ##니다 . [SEP]'

In [14]:
data2[1]

[0,
 1385,
 1535,
 543,
 4872,
 5851,
 1504,
 1415,
 2062,
 594,
 930,
 187,
 3606,
 31,
 31,
 2,
 1443,
 16,
 12700,
 2255,
 5851,
 1504,
 1415,
 2062,
 594,
 1889,
 187,
 3606,
 18,
 2]

In [15]:
''.join(tokenizer.batch_decode(data2[0][:150]))

'[CLS]스릴도있고반전도있고여느한국영화쓰레기들하고는차원이다르네요~[SEP]반전도있고,사랑도있고재미도있네요.[SEP]'

In [20]:
samp_pd['bert_tokenized']=pd.Series(map(lambda x : ''.join(tokenizer.batch_decode(x)),data))
samp_pd['kkma_tokenized']=pd.Series(map(lambda x : ' '.join(tokenizer.batch_decode(x)),data2))

c:\Users\kzvxy\.conda\envs\test\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\Users\kzvxy\.conda\envs\test\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
samp_pd

,sentence_1,sentence_2,sentence_a,label,pos_sen_1,pos_sen_2,bert_tokenized,kkma_tokenized
0,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.","스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~ 반전도 있고,...",2.2,스릴 도 있 고 반전 도 있 고 여느 한국 영화 쓰레기 들 하 고는 차원 이 다르 ...,"반전 도 있 고 , 사랑 도 있 고 재미 도 있 네요 .",[CLS]스릴##도##있##고반전##도있##고여느한국##영화쓰레기##들##하고##는...,[CLS] 스릴 도 있 고 반전 도 있 고 여느 한국 영화 쓰레기 들 하 고는 차원...
1,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.","앗 제가 접근권한이 없다고 뜹니다;; 오, 액세스 권한이 없다고 합니다.",4.2,앗 저 가 접근 권한 이 없다 고 뜨 ㅂ니다 ;;,"오 , 액세스 권한 이 없다 고 하 ㅂ니다 .","[CLS]앗제##가접근##권##한##이없##다고[UNK];;[SEP]오,액세##스권...",[CLS] 앗 저 가 접근 권한 이 없 ##다 고 뜨 ㅂ ##니다 ; ; [SEP]...
2,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,주택청약조건 변경해주세요. 주택청약 무주택기준 변경해주세요.,2.4,주택 청약 조건 변경 하 어 주 세요 .,주택 청약 무 주택 기준 변경 하 어 주 세요 .,[CLS]주택##청##약##조건변경##해##주##세요.[SEP]주택##청##약무주택...,[CLS] 주택 청약 조건 변경 하 어 주 세요 . [SEP] 주택 청약 무 주택 ...
3,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,입사후 처음 대면으로 만나 반가웠습니다. 화상으로만 보다가 리얼로 만나니 정말 반가...,3.0,입사 후 처음 대면 으로 만 나 반갑 었 습니다 .,화상 으로 만 보 다가 리얼 로 만나 니 정말 반갑 었 습니다 .,[CLS]입사##후처음대면##으로만나반가웠##습##니다.[SEP]화상##으로##만보...,[CLS] 입사 후 처음 대면 으로 만 나 반갑 었 습니다 . [SEP] 화상 으로...
4,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,뿌듯뿌듯 하네요!! 꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,뿌듯 뿌듯 하 네요 !!,꼬옥 실제로 한번 뵈 어요 뿌뿌뿌 ~ ! ~ !,[CLS]뿌듯##뿌##듯하##네##요!![SEP]꼬##옥실제로한번뵈##어요뿌##뿌#...,[CLS] 뿌듯 뿌듯 하 네요 ! ! [SEP] 꼬 ##옥 실제로 한번 뵈 어요 뿌...
...,...,...,...,...,...,...,...,...
9319,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다 본인이 납부한 국...,0.2,교원 능력 개발 평가 에서 교원 이 보호 받 을 수 있 는 장치 를 마련 하 어야 ...,본인 이 납부 하 ㄴ 국민 연금 금액 을 기준 으로 대출 을 받 을 수 있 는 제도...,[CLS]교원##능력##개발##평가##에서교원##이보호##받##을수있##는장치##를...,[CLS] 교원 능력 개발 평가 에서 교원 이 보호 받 을 수 있 는 장치 를 마련...
9320,여성가족부의 폐지를 원합니드,여성가족부 폐지를 청원 합니다.,여성가족부의 폐지를 원합니드 여성가족부 폐지를 청원 합니다.,4.2,여성 가족 부의 폐지 를 원 합 니 드,여성 가족 부 폐지 를 청원 하 ㅂ니다 .,[CLS]여성##가족##부##의폐지##를원##합##니##드[SEP]여성##가족##부...,[CLS] 여성 가족 부의 폐지 를 원 합 니 드 [SEP] 여성 가족 부 폐지 를...
9321,국회의원들 월급좀 줄여주세요,공무원 봉급좀 줄이지좀 마세요,국회의원들 월급좀 줄여주세요 공무원 봉급좀 줄이지좀 마세요,0.6,국회의원 들 월급 좀 줄이 어 주 세요,공무원 봉급 좀 줄이 지 좀 마 세요,[CLS]국회의원##들월급##좀줄여##주##세요[SEP]공무원봉급##좀줄이##지##...,[CLS] 국회의원 들 월급 좀 줄이 어 주 세요 [SEP] 공무원 봉급 좀 줄이 ...
9322,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!!,오늘 못먹은 밥은 꼭 담에 먹기로 하고요!!,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!! 오늘 못먹은 밥은 꼭 담에 먹...,3.2,오늘 못하 ㄴ 점심 은 다음 에 다시 츄 라이 하 기로 해 요 !!,오늘 못 먹 은 밥 은 꼭 담 에 먹 기로 하 고요 !!,[CLS]오늘못한점심##은다음##에다시츄##라이하##기##로해요!![SEP]오늘못#...,[CLS] 오늘 못하 ㄴ 점심 은 다음 에 다시 츄 라이 하 기로 해 요 ! ! [...


In [23]:
samp_pd[['bert_tokenized','kkma_tokenized']].head(15)

,bert_tokenized,kkma_tokenized
0,[CLS]스릴##도##있##고반전##도있##고여느한국##영화쓰레기##들##하고##는...,[CLS] 스릴 도 있 고 반전 도 있 고 여느 한국 영화 쓰레기 들 하 고는 차원...
1,"[CLS]앗제##가접근##권##한##이없##다고[UNK];;[SEP]오,액세##스권...",[CLS] 앗 저 가 접근 권한 이 없 ##다 고 뜨 ㅂ ##니다 ; ; [SEP]...
2,[CLS]주택##청##약##조건변경##해##주##세요.[SEP]주택##청##약무주택...,[CLS] 주택 청약 조건 변경 하 어 주 세요 . [SEP] 주택 청약 무 주택 ...
3,[CLS]입사##후처음대면##으로만나반가웠##습##니다.[SEP]화상##으로##만보...,[CLS] 입사 후 처음 대면 으로 만 나 반갑 었 습니다 . [SEP] 화상 으로...
4,[CLS]뿌듯##뿌##듯하##네##요!![SEP]꼬##옥실제로한번뵈##어요뿌##뿌#...,[CLS] 뿌듯 뿌듯 하 네요 ! ! [SEP] 꼬 ##옥 실제로 한번 뵈 어요 뿌...
5,[CLS][UNK][SEP]오마이갓지저##스스크##론이스트팬[SEP],[CLS] 오 마이 가 뜨 지 지 어 [UNK] 이스트 [UNK] [SEP] 오 마...
6,[CLS]전암##만찍##어도까만하늘..ㅠㅠ[SEP]암##만찍##어도하늘##은까맣##...,[CLS] 전 암 ##만 찍 어도 까맣 ㄴ 하늘 . . ㅠㅠ [SEP] 암 ##만 ...
7,[CLS]이렇게귀여운쥐##들##은처음##이##네##요.ㅎㅎㅎ[SEP]이렇게지##겨운...,[CLS] 이렇 게 귀엽 ㄴ 쥐 들 은 처음 이 네요 . ㅎㅎㅎ [SEP] 이렇 게...
8,[CLS]미세먼지해결##이가장시급##한문제##입니다![SEP]가장시급##한것##이신...,[CLS] 미세 먼지 해결 이 가장 시급 하 ㄴ 문제 이 ㅂ ##니다 ! [SEP]...
9,[CLS]크림##하우스환불##조치##해##주##세요.[SEP]크림##하우스환불##조...,[CLS] 크림 하우스 환불 조치 하 어 주 세요 . [SEP] 크림 하우스 환불 ...


In [25]:
# samp_pd[['bert_tokenized','kkma_tokenized']].to_csv('../../../../data/pos_compare.csv')

In [26]:
from hanspell import spell_checker

sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "
spelled_sent = spell_checker.check(sent)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)

맞춤법 틀리면 왜 안돼? 쓰고 싶은 대로 쓰면 되지


In [32]:
samp_pd['sentence_1'][0]

'스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~'

In [34]:
spelled_sent = spell_checker.check(samp_pd['sentence_1'][0])
hanspell_sent = spelled_sent.checked
print(hanspell_sent)

스릴도 있고 반전도 있고 여느 한국 영화 쓰레기들하고는 차원이 다르네요~


In [36]:
' '.join(kkma.morphs(hanspell_sent))

'스릴 도 있 고 반전 도 있 고 여느 한국 영화 쓰레기 들 하 고는 차원 이 다르 네요 ~'

In [37]:
dl1 = []
dl2 = []
for i in range(len(samp_pd)):
    try:
        spelled_sent = spell_checker.check(samp_pd['sentence_1'][i])
        hanspell_sent = spelled_sent.checked
        dl1.append(' '.join(kkma.morphs(hanspell_sent)))
    except:
        dl1.append(samp_pd['sentence_1'][i])
    try:
        spelled_sent2 = spell_checker.check(samp_pd['sentence_2'][i])
        hanspell_sent2 = spelled_sent2.checked
        dl2.append(' '.join(kkma.morphs(hanspell_sent2)))
    except:
        dl2.append(samp_pd['sentence_2'][i])
samp_pd['cl_sen_1'] = dl1
samp_pd['cl_sen_2'] = dl2

c:\Users\kzvxy\.conda\envs\test\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
c:\Users\kzvxy\.conda\envs\test\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [38]:
samp_pd['cl_sen_1']

0       스릴 도 있 고 반전 도 있 고 여느 한국 영화 쓰레기 들 하 고는 차원 이 다르 ...
1                             앗 저 가 접근 권한 이 없다 고 뜨 ㅂ니다 ;;
2                                  주택 청약 조건 변경 하 어 주 세요 .
3                            입사 후 처음 대면 으로 만 나 반갑 었 습니다 .
4                                           뿌듯 뿌듯 하 네요 !!
                              ...                        
9319    교원 능력 개발 평가 에서 교원 이 보호 받 을 수 있 는 장치 를 마련 하 어야 ...
9320                                여성 가족 부의 폐지 를 원 합 니 드
9321                                국회의원 들 월급 좀 줄이 어 주 세요
9322                 오늘 못하 ㄴ 점심 은 다음 에 다시 트라이 하 기로 해 요 !!
9323                                 법정 공휴일 휴무 관련 ( 근로자 )
Name: cl_sen_1, Length: 9324, dtype: object

In [39]:
df = samp_pd
data3 = []
for idx, item in tqdm(df.iterrows(), desc='tokenizing', total=len(df)):
# 두 입력 문장을 [SEP] 토큰으로 이어붙여서 전처리합니다.
    text = '[SEP]'.join([item[text_column] for text_column in ['cl_sen_1', 'cl_sen_2']])
    outputs = tokenizer(text, add_special_tokens=True, padding='do_not_pad', truncation=True)
    data3.append(outputs['input_ids'])

tokenizing:   0%|          | 0/9324 [00:00<?, ?it/s]

In [41]:
samp_pd['cls_tokenized']=pd.Series(map(lambda x : ' '.join(tokenizer.batch_decode(x)),data3))

c:\Users\kzvxy\.conda\envs\test\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [42]:
samp_pd

,sentence_1,sentence_2,sentence_a,label,pos_sen_1,pos_sen_2,bert_tokenized,kkma_tokenized,cl_sen_1,cl_sen_2,cls_tokenized
0,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.","스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~ 반전도 있고,...",2.2,스릴 도 있 고 반전 도 있 고 여느 한국 영화 쓰레기 들 하 고는 차원 이 다르 ...,"반전 도 있 고 , 사랑 도 있 고 재미 도 있 네요 .",[CLS]스릴##도##있##고반전##도있##고여느한국##영화쓰레기##들##하고##는...,[CLS] 스릴 도 있 고 반전 도 있 고 여느 한국 영화 쓰레기 들 하 고는 차원...,스릴 도 있 고 반전 도 있 고 여느 한국 영화 쓰레기 들 하 고는 차원 이 다르 ...,"반전 도 있 고 , 사랑 도 있 고 재미 도 있 네요 .",[CLS] 스릴 도 있 고 반전 도 있 고 여느 한국 영화 쓰레기 들 하 고는 차원...
1,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.","앗 제가 접근권한이 없다고 뜹니다;; 오, 액세스 권한이 없다고 합니다.",4.2,앗 저 가 접근 권한 이 없다 고 뜨 ㅂ니다 ;;,"오 , 액세스 권한 이 없다 고 하 ㅂ니다 .","[CLS]앗제##가접근##권##한##이없##다고[UNK];;[SEP]오,액세##스권...",[CLS] 앗 저 가 접근 권한 이 없 ##다 고 뜨 ㅂ ##니다 ; ; [SEP]...,앗 저 가 접근 권한 이 없다 고 뜨 ㅂ니다 ;;,"오 , 액세스 권한 이 없다 고 하 ㅂ니다 .",[CLS] 앗 저 가 접근 권한 이 없 ##다 고 뜨 ㅂ ##니다 ; ; [SEP]...
2,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,주택청약조건 변경해주세요. 주택청약 무주택기준 변경해주세요.,2.4,주택 청약 조건 변경 하 어 주 세요 .,주택 청약 무 주택 기준 변경 하 어 주 세요 .,[CLS]주택##청##약##조건변경##해##주##세요.[SEP]주택##청##약무주택...,[CLS] 주택 청약 조건 변경 하 어 주 세요 . [SEP] 주택 청약 무 주택 ...,주택 청약 조건 변경 하 어 주 세요 .,주택 청약 무 주택 기준 변경 하 어 주 세요 .,[CLS] 주택 청약 조건 변경 하 어 주 세요 . [SEP] 주택 청약 무 주택 ...
3,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,입사후 처음 대면으로 만나 반가웠습니다. 화상으로만 보다가 리얼로 만나니 정말 반가...,3.0,입사 후 처음 대면 으로 만 나 반갑 었 습니다 .,화상 으로 만 보 다가 리얼 로 만나 니 정말 반갑 었 습니다 .,[CLS]입사##후처음대면##으로만나반가웠##습##니다.[SEP]화상##으로##만보...,[CLS] 입사 후 처음 대면 으로 만 나 반갑 었 습니다 . [SEP] 화상 으로...,입사 후 처음 대면 으로 만 나 반갑 었 습니다 .,화상 으로 만 보 다가 리얼 로 만나 니 정말 반갑 었 습니다 .,[CLS] 입사 후 처음 대면 으로 만 나 반갑 었 습니다 . [SEP] 화상 으로...
4,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,뿌듯뿌듯 하네요!! 꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,뿌듯 뿌듯 하 네요 !!,꼬옥 실제로 한번 뵈 어요 뿌뿌뿌 ~ ! ~ !,[CLS]뿌듯##뿌##듯하##네##요!![SEP]꼬##옥실제로한번뵈##어요뿌##뿌#...,[CLS] 뿌듯 뿌듯 하 네요 ! ! [SEP] 꼬 ##옥 실제로 한번 뵈 어요 뿌...,뿌듯 뿌듯 하 네요 !!,꼬옥 실제로 한번 뵈 어요 뿔 뿌 푸 ~ ! ~ !,[CLS] 뿌듯 뿌듯 하 네요 ! ! [SEP] 꼬 ##옥 실제로 한번 뵈 어요 뿔...
...,...,...,...,...,...,...,...,...,...,...,...
9319,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다 본인이 납부한 국...,0.2,교원 능력 개발 평가 에서 교원 이 보호 받 을 수 있 는 장치 를 마련 하 어야 ...,본인 이 납부 하 ㄴ 국민 연금 금액 을 기준 으로 대출 을 받 을 수 있 는 제도...,[CLS]교원##능력##개발##평가##에서교원##이보호##받##을수있##는장치##를...,[CLS] 교원 능력 개발 평가 에서 교원 이 보호 받 을 수 있 는 장치 를 마련...,교원 능력 개발 평가 에서 교원 이 보호 받 을 수 있 는 장치 를 마련 하 어야 ...,본인 이 납부 하 ㄴ 국민 연금 금액 을 기준 으로 대출 을 받 을 수 있 는 제도...,[CLS] 교원 능력 개발 평가 에서 교원 이 보호 받 을 수 있 는 장치 를 마련...
9320,여성가족부의 폐지를 원합니드,여성가족부 폐지를 청원 합니다.,여성가족부의 폐지를 원합니드 여성가족부 폐지를 청원 합니다.,4.2,여성 가족 부의 폐지 를 원 합 니 드,여성 가족 부 폐지 를 청원 하 ㅂ니다 .,[CLS]여성##가족##부##의폐지##를원##합##니##드[SEP]여성##가족##부...,[CLS] 여성 가족 부의 폐지 를 원 합 니 드 [SEP] 여성 가족 부 폐지 를...,여성 가족 부의 폐지 를 원 합 니 드,여성 가족 부 폐지 를 청원 하 ㅂ니다 .,[CLS] 여성 가족 부의 폐지 를 원 합 니 드 [SEP] 여성 가족 부 폐지 를...
9321,국회의원들 월급좀 줄여주세요,공무원 봉급좀 줄이지좀 마세요,국회의원들 월급좀 줄여주세요 공무원 봉급좀 줄이지좀 마세요,0.6,국회의원 들 월급 좀 줄이 어 주 세요,공무원 봉급 좀 줄이 지 좀 마 세요,[CLS]국회의원##들월급##좀줄여##주##세요[SEP]공무원봉급##좀줄이##지##...,[CLS] 국회의원 들 월급 좀 줄이 어 주 세요 [SEP] 공무원 봉급 좀 줄이 ...,국회의원 들 월급 좀 줄이 어 주 세요,공무원 봉급 좀 줄이 지 좀 마 세요,[CLS] 국회의원 들 월급 좀 줄이 어 주 세요 [SEP] 공무원 봉급 좀 줄이 ...
9322,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!!,오늘 못먹은 밥은 꼭 담에 먹기로 하고요!!,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!! 오늘 못먹은 밥은 꼭 담에 먹...,3.2,오늘 못하 ㄴ 점심 은 다음 에 다시 츄 라이 하 기로 해 요 !!,오늘 못 먹 은 밥 은 꼭 담 에 먹 기로 하 고요 !!,[CLS]오늘못한점심##은다음##에다시츄##라이하##기##로해요!![SEP]오늘못#...,[CLS] 오늘 못하 ㄴ 점심 은 다음 에 다시 츄 라이 하 기로 해 요 ! ! [...,오늘 못하 ㄴ 점심 은 다음 에 다시 트라이 하 기로 해 요 !!,오늘 못 먹 은 밥 은 꼭 담 에 먹 기로 하 고요 !!,[CLS] 오늘 못하 ㄴ 점심 은 다음 에 다시 트라이 하 기로 해 요 ! ! [S...


In [43]:
dl1 = []
dl2 = []
for i in range(len(samp_pd)):
    try:
        spelled_sent = spell_checker.check(samp_pd['sentence_1'][i])
        hanspell_sent = spelled_sent.checked
        dl1.append(' '.join(kkma.morphs(hanspell_sent)))
    except:
        dl1.append(samp_pd['sentence_1'][i])
    try:
        spelled_sent2 = spell_checker.check(samp_pd['sentence_2'][i])
        hanspell_sent2 = spelled_sent2.checked
        dl2.append(' '.join(kkma.morphs(hanspell_sent2)))
    except:
        dl2.append(samp_pd['sentence_2'][i])
samp_pd['sentence_1'] = dl1
samp_pd['sentence_2'] = dl2

c:\Users\kzvxy\.conda\envs\test\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
c:\Users\kzvxy\.conda\envs\test\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [44]:
samp_pd

,sentence_1,sentence_2,sentence_a,label,pos_sen_1,pos_sen_2,bert_tokenized,kkma_tokenized,cl_sen_1,cl_sen_2,cls_tokenized
0,스릴 도 있 고 반전 도 있 고 여느 한국 영화 쓰레기 들 하 고는 차원 이 다르 ...,"반전 도 있 고 , 사랑 도 있 고 재미 도 있 네요 .","스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~ 반전도 있고,...",2.2,스릴 도 있 고 반전 도 있 고 여느 한국 영화 쓰레기 들 하 고는 차원 이 다르 ...,"반전 도 있 고 , 사랑 도 있 고 재미 도 있 네요 .",[CLS]스릴##도##있##고반전##도있##고여느한국##영화쓰레기##들##하고##는...,[CLS] 스릴 도 있 고 반전 도 있 고 여느 한국 영화 쓰레기 들 하 고는 차원...,스릴 도 있 고 반전 도 있 고 여느 한국 영화 쓰레기 들 하 고는 차원 이 다르 ...,"반전 도 있 고 , 사랑 도 있 고 재미 도 있 네요 .",[CLS] 스릴 도 있 고 반전 도 있 고 여느 한국 영화 쓰레기 들 하 고는 차원...
1,앗 저 가 접근 권한 이 없다 고 뜨 ㅂ니다 ;;,"오 , 액세스 권한 이 없다 고 하 ㅂ니다 .","앗 제가 접근권한이 없다고 뜹니다;; 오, 액세스 권한이 없다고 합니다.",4.2,앗 저 가 접근 권한 이 없다 고 뜨 ㅂ니다 ;;,"오 , 액세스 권한 이 없다 고 하 ㅂ니다 .","[CLS]앗제##가접근##권##한##이없##다고[UNK];;[SEP]오,액세##스권...",[CLS] 앗 저 가 접근 권한 이 없 ##다 고 뜨 ㅂ ##니다 ; ; [SEP]...,앗 저 가 접근 권한 이 없다 고 뜨 ㅂ니다 ;;,"오 , 액세스 권한 이 없다 고 하 ㅂ니다 .",[CLS] 앗 저 가 접근 권한 이 없 ##다 고 뜨 ㅂ ##니다 ; ; [SEP]...
2,주택 청약 조건 변경 하 어 주 세요 .,주택 청약 무 주택 기준 변경 하 어 주 세요 .,주택청약조건 변경해주세요. 주택청약 무주택기준 변경해주세요.,2.4,주택 청약 조건 변경 하 어 주 세요 .,주택 청약 무 주택 기준 변경 하 어 주 세요 .,[CLS]주택##청##약##조건변경##해##주##세요.[SEP]주택##청##약무주택...,[CLS] 주택 청약 조건 변경 하 어 주 세요 . [SEP] 주택 청약 무 주택 ...,주택 청약 조건 변경 하 어 주 세요 .,주택 청약 무 주택 기준 변경 하 어 주 세요 .,[CLS] 주택 청약 조건 변경 하 어 주 세요 . [SEP] 주택 청약 무 주택 ...
3,입사 후 처음 대면 으로 만 나 반갑 었 습니다 .,화상 으로 만 보 다가 리얼 로 만나 니 정말 반갑 었 습니다 .,입사후 처음 대면으로 만나 반가웠습니다. 화상으로만 보다가 리얼로 만나니 정말 반가...,3.0,입사 후 처음 대면 으로 만 나 반갑 었 습니다 .,화상 으로 만 보 다가 리얼 로 만나 니 정말 반갑 었 습니다 .,[CLS]입사##후처음대면##으로만나반가웠##습##니다.[SEP]화상##으로##만보...,[CLS] 입사 후 처음 대면 으로 만 나 반갑 었 습니다 . [SEP] 화상 으로...,입사 후 처음 대면 으로 만 나 반갑 었 습니다 .,화상 으로 만 보 다가 리얼 로 만나 니 정말 반갑 었 습니다 .,[CLS] 입사 후 처음 대면 으로 만 나 반갑 었 습니다 . [SEP] 화상 으로...
4,뿌듯 뿌듯 하 네요 !!,꼬옥 실제로 한번 뵈 어요 뿔 뿌 푸 ~ ! ~ !,뿌듯뿌듯 하네요!! 꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,뿌듯 뿌듯 하 네요 !!,꼬옥 실제로 한번 뵈 어요 뿌뿌뿌 ~ ! ~ !,[CLS]뿌듯##뿌##듯하##네##요!![SEP]꼬##옥실제로한번뵈##어요뿌##뿌#...,[CLS] 뿌듯 뿌듯 하 네요 ! ! [SEP] 꼬 ##옥 실제로 한번 뵈 어요 뿌...,뿌듯 뿌듯 하 네요 !!,꼬옥 실제로 한번 뵈 어요 뿔 뿌 푸 ~ ! ~ !,[CLS] 뿌듯 뿌듯 하 네요 ! ! [SEP] 꼬 ##옥 실제로 한번 뵈 어요 뿔...
...,...,...,...,...,...,...,...,...,...,...,...
9319,교원 능력 개발 평가 에서 교원 이 보호 받 을 수 있 는 장치 를 마련 하 어야 ...,본인 이 납부 하 ㄴ 국민 연금 금액 을 기준 으로 대출 을 받 을 수 있 는 제도...,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다 본인이 납부한 국...,0.2,교원 능력 개발 평가 에서 교원 이 보호 받 을 수 있 는 장치 를 마련 하 어야 ...,본인 이 납부 하 ㄴ 국민 연금 금액 을 기준 으로 대출 을 받 을 수 있 는 제도...,[CLS]교원##능력##개발##평가##에서교원##이보호##받##을수있##는장치##를...,[CLS] 교원 능력 개발 평가 에서 교원 이 보호 받 을 수 있 는 장치 를 마련...,교원 능력 개발 평가 에서 교원 이 보호 받 을 수 있 는 장치 를 마련 하 어야 ...,본인 이 납부 하 ㄴ 국민 연금 금액 을 기준 으로 대출 을 받 을 수 있 는 제도...,[CLS] 교원 능력 개발 평가 에서 교원 이 보호 받 을 수 있 는 장치 를 마련...
9320,여성 가족 부의 폐지 를 원 합 니 드,여성 가족 부 폐지 를 청원 하 ㅂ니다 .,여성가족부의 폐지를 원합니드 여성가족부 폐지를 청원 합니다.,4.2,여성 가족 부의 폐지 를 원 합 니 드,여성 가족 부 폐지 를 청원 하 ㅂ니다 .,[CLS]여성##가족##부##의폐지##를원##합##니##드[SEP]여성##가족##부...,[CLS] 여성 가족 부의 폐지 를 원 합 니 드 [SEP] 여성 가족 부 폐지 를...,여성 가족 부의 폐지 를 원 합 니 드,여성 가족 부 폐지 를 청원 하 ㅂ니다 .,[CLS] 여성 가족 부의 폐지 를 원 합 니 드 [SEP] 여성 가족 부 폐지 를...
9321,국회의원 들 월급 좀 줄이 어 주 세요,공무원 봉급 좀 줄이 지 좀 마 세요,국회의원들 월급좀 줄여주세요 공무원 봉급좀 줄이지좀 마세요,0.6,국회의원 들 월급 좀 줄이 어 주 세요,공무원 봉급 좀 줄이 지 좀 마 세요,[CLS]국회의원##들월급##좀줄여##주##세요[SEP]공무원봉급##좀줄이##지##...,[CLS] 국회의원 들 월급 좀 줄이 어 주 세요 [SEP] 공무원 봉급 좀 줄이 ...,국회의원 들 월급 좀 줄이 어 주 세요,공무원 봉급 좀 줄이 지 좀 마 세요,[CLS] 국회의원 들 월급 좀 줄이 어 주 세요 [SEP] 공무원 봉급 좀 줄이 ...
9322,오늘 못하 ㄴ 점심 은 다음 에 다시 트라이 하 기로 해 요 !!,오늘 못 먹 은 밥 은 꼭 담 에 먹 기로 하 고요 !!,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!! 오늘 못먹은 밥은 꼭 담에 먹...,3.2,오늘 못하 ㄴ 점심 은 다음 에 다시 츄 라이 하 기로 해 요 !!,오늘 못 먹 은 밥 은 꼭 담 에 먹 기로 하 고요 !!,[CLS]오늘못한점심##은다음##에다시츄##라이하##기##로해요!![SEP]오늘못#...,[CLS] 오늘 못하 ㄴ 점심 은 다음 에 다시 츄 라이 하 기로 해 요 ! ! [...,오늘 못하 ㄴ 점심 은 다음 에 다시 트라이 하 기로 해 요 !!,오늘 못 먹 은 밥 은 꼭 담 에 먹 기로 하 고요 !!,[CLS] 오늘 못하 ㄴ 점심 은 다음 에 다시 트라이 하 기로 해 요 ! ! [S...
